In [10]:
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
# set up auto load
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from load_data_multipred import *

plt.rcParams['font.family'] = 'serif'
sns.set_style("darkgrid")

if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"
print('PyTorch CUDA is available?', torch.cuda.is_available())

PyTorch CUDA is available? True


In [13]:
train_ticker_strings = []
for filename in os.listdir('price'):
  if filename != 'MSFT.csv':
    train_ticker_strings.append(filename[:-4])
print(train_ticker_strings)
#train_ticker_strings = ["AAPL", "ABB", "ABBV", "AEP", "AGFS", "AMGN", "AMZN", "BA", "BABA", "BAC", "BBL", "BCH", "BHP", "BP", "BRK-A", "BSAC", "BUD", "C", "CAT", "CELG", "CHL", "CHTR", "CMCSA", "CODI", "CSCO", "CVX", "D", "FB", "PEP", "ORCL", "UPS", "GOOG", "KO", "JPM", "PCG", ]
look_back = 30
train_data_loader = get_data_loader(TransformerMultiFeaturesDataset, train_ticker_strings, look_back=look_back)
print("train data size (in batches):", len(train_data_loader))

['MMM', 'CVX', 'HRG', 'MRK', 'PICO', 'CAT', 'AGFS', 'PM', 'PPL', 'JPM', 'TSM', 'ABB', 'PCLN', 'AEP', 'BCH', 'CSCO', 'C', 'WMT', 'V', 'UPS', 'CHTR', 'TOT', 'BABA', 'D', 'GMRE', 'CODI', 'NVS', 'AAPL', 'HON', 'BBL', 'MDT', 'SLB', 'MA', 'SPLP', 'ORCL', 'KO', 'GOOG', 'BA', 'TM', 'UNH', 'MCD', 'FB', 'AMGN', 'NEE', 'NGG', 'IEP', 'T', 'CELG', 'UTX', 'XOM', 'REX', 'DIS', 'BRK-A', 'MO', 'GD', 'DHR', 'UL', 'HD', 'PEP', 'LMT', 'SNP', 'ABBV', 'SNY', 'BSAC', 'UN', 'AMZN', 'INTC', 'BHP', 'CHL', 'JNJ', 'PTR', 'PFE', 'BUD', 'VZ', 'SRE', 'GE', 'DUK', 'BAC', 'HSBC', 'BP', 'EXC', 'PG', 'SO', 'CMCSA', 'WFC', 'PCG', 'RDS-B']
train data size (in batches): 1946


In [14]:
train_small_data_loader = get_data_loader(TransformerMultiFeaturesDataset, train_ticker_strings[:1], look_back=look_back)
print("train small data size (in batches):", len(train_small_data_loader))

train small data size (in batches): 18


In [15]:
from linearmodel_multiplepred import *

model = Model(num_pred=5)
model = model.to(device)

# w_loss = 1
lr = 0.01
loss_fn_mse = torch.nn.MSELoss()
# loss_fn_bin = torch.nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=lr)

print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

Model(
  (Linear): ModuleList(
    (0): Linear(in_features=30, out_features=5, bias=True)
  )
)
2
torch.Size([5, 30])
torch.Size([5])


In [17]:
input_dim = 4
hidden_dim = 256
num_layers = 6
output_dim = 1
num_epochs = 50
lr = 0.01
dim_feedforward = 1024
nhead = 2
dropout=0.1
hist = np.zeros(num_epochs)
w_loss = 0
# try model on small dataset
for t in range(num_epochs):
    total_loss = 0
    total_mse = 0
    total_bin = 0
    # y_train_preds = []
    for X_train, y_train in train_small_data_loader:
        X_train = X_train.to(device)
        y_train = y_train.to(device)

        # Forward pass
        y_train_pred = model(X_train)
        # # binarize pred and y for binary loss func
        # y_train_pred_bin = y_train_pred.clone()
        # y_train_pred_bin[(y_train_pred-X_train[:,-1:,:1])>0] = 1
        # y_train_pred_bin[(y_train_pred-X_train[:,-1:,:1])<=0] = 0
        # y_train_bin = y_train.clone()
        
        # y_train_bin[(y_train-X_train[:,-1:,0])>0] = 1
        # y_train_bin[(y_train-X_train[:,-1:,0])<=0] = 0

        # y_train_preds.extend(y_train_pred)
        print(y_train_pred.shape, y_train.shape)
        loss_mse = loss_fn_mse(y_train_pred.flatten(), y_train.flatten().to(device)) 
        # loss_bin = w_loss*loss_fn_bin(y_train_pred_bin.flatten(), y_train_bin.to(device).flatten())
        #print("!!!!!", loss_bin)
        loss = loss_mse

        total_mse += loss_mse.item()
        total_loss += loss_mse.item()
        
        # Zero out gradient, else they will accumulate between epochs
        optimiser.zero_grad()

        # Backward pass
        loss.backward()
        # Update parameters
        optimiser.step()

    hist[t] = total_loss/len(train_small_data_loader) 
    if t % 10 == 0 and t !=0:
        print("Epoch ", t, "Total Loss: ", total_loss, "MSE Loss: ", total_mse, "Binary Loss: ", total_bin)

torch.Size([32, 5, 4]) torch.Size([32, 1])


RuntimeError: ignored